In [1]:
# loading dependencies
!pip install accelerate==0.27.2 --progress-bar off
!pip install -q peft==0.4.0 --progress-bar off
# !pip install -q bitsandbytes==0.40.2 --progress-bar off
!pip install -q bitsandbytes==0.41.3 --progress-bar off
!pip install -q transformers==4.38.2 --progress-bar off
!pip install -q trl==0.4.7 --progress-bar off

In [2]:
import os
from random import randrange
from functools import partial
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoModelForSequenceClassification,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          DataCollatorWithPadding,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM, AutoPeftModelForSequenceClassification, TaskType
from trl import SFTTrainer

2024-03-26 22:10:06.286333: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 22:10:06.286448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 22:10:06.424716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from imblearn.metrics import geometric_mean_score
import numpy as np
import pandas as pd

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [26]:
model_name = "meta-llama/Llama-2-7b-hf"
output_dir = "/kaggle/working/results_class_e1_bce_7layers"

In [26]:
os.rename(output_dir,"/kaggle/working/results_class_e1_bce_1layer")

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/results_class_e1_bce_all_layers' -> '/kaggle/working/results_class_e1_bce_1layer'

In [29]:
from huggingface_hub import HfApi
api = HfApi()

# Upload all the content from the local folder to your remote Space.
# By default, files are uploaded at the root of the repo
api.upload_folder(
    folder_path=output_dir,
    repo_id="FakeNewsLlama/CrimeTrueFake7Layers",
)

events.out.tfevents.1711491443.bd9d44df5192.34.0:   0%|          | 0.00/6.18k [00:00<?, ?B/s]

adapter_model.bin:   0%|          | 0.00/35.0M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/FakeNewsLlama/CrimeTrueFake7Layers/commit/3857e40cef0a2435ba1dd592cd68302008f211b2', commit_message='Upload folder using huggingface_hub', commit_description='', oid='3857e40cef0a2435ba1dd592cd68302008f211b2', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# only to remove directories
import shutil
# shutil.rmtree('/kaggle/working/news_classification_llama2_7b')
# shutil.rmtree('/kaggle/working/results_class_e1_bce')

In [6]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
    """
    Configures model quantization method using bitsandbytes to speed up training and inference

    :param load_in_4bit: Load model in 4-bit precision mode
    :param bnb_4bit_use_double_quant: Nested quantization for 4-bit model
    :param bnb_4bit_quant_type: Quantization data type for 4-bit model
    :param bnb_4bit_compute_dtype: Computation data type for 4-bit model
    """

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config

In [7]:
# Activate 4-bit precision base model loading
load_in_4bit = True

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute data type for 4-bit base models
bnb_4bit_compute_dtype = torch.bfloat16

bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

*loading model and tokenizer*

In [8]:
# loading model and tokenizer
n_gpus = torch.cuda.device_count()
max_memory = f'{40960}MB'

label2id = {'TRUE': 0, 'FAKE': 1}
id2label = {0: 'TRUE', 1: 'FAKE'}

#load model
model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
        num_labels = 2,
        label2id = label2id,
        id2label = id2label
    )

model.config.pad_token_id = model.config.eos_token_id
model.config.problem_type = "single_label_classification"

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
#load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = True)
tokenizer.pad_token = tokenizer.eos_token

/opt/conda/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [12]:
for name, module in model.named_modules():
    print(name)


model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.rotary_emb
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.rotary_emb
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.

In [14]:
find_all_linear_names(model)

model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.2.self_attn.q_proj
model.layers.2.self_attn.k_proj
model.layers.2.self_attn.v_proj
model.layers.2.self_attn.o_proj
model.layers.2.mlp.gate_proj
model.layers.2.mlp.up_proj
model.layers.2.mlp.down_proj
model.layers.3.self_attn.q_proj
model.layers.3.self_attn.k_proj
model.layers.3.self_attn.v_proj
model.layers.3.self_attn.o_proj
model.layers.3.mlp.gate_proj
model.layers.3.mlp.up_proj
model.layers.3.mlp.down_proj
model.layers.4.self_attn.q_proj
model.layers.4.self_attn.k_proj
model.layers.4.self_attn.v_proj
model.layers.4.self_attn.o_proj
model.layers.4.mlp.g

['down_proj', 'v_proj', 'gate_proj', 'up_proj', 'k_proj', 'o_proj', 'q_proj']

In [ ]:
from torchinfo import summary
summary(model)

In [10]:
# Loading train data

train_df = pd.read_csv("/kaggle/input/crime-split/train_crime_df.csv")
display(train_df['label'].value_counts())

target_map = { 'TRUE': 0, 'FAKE': 1 }
target_map_hot = { 'TRUE': [1,0], 'FAKE': [0,1] }
train_df['target'] = train_df['label'].map(target_map)

train_df = train_df.drop(['label'], axis=1)
train_df = train_df.rename(columns={'target': 'labels'})

display(train_df.dtypes)
display(train_df.head())

label
TRUE    341
FAKE    302
Name: count, dtype: int64

Unnamed: 0     int64
text          object
metadata      object
domain        object
labels         int64
dtype: object

,Unnamed: 0,text,metadata,domain,labels
0,283,st by security HQ in central Syria NGO AFP Thu...,"[{'article': None, 'author': None, 'date': '3/...",CRIME,1
1,163,1 civilians in east Syria Monitor AFP Friday 1...,"[{'article': None, 'author': None, 'date': '7/...",CRIME,1
2,681,ort URL 0 15 The death toll following Daesh Th...,"[{'article': None, 'author': None, 'date': '5/...",CRIME,1
3,555,slamic State beheads crucifies in push for Syr...,"[{'article': None, 'author': None, 'date': '8/...",CRIME,0
4,259,men in Syrias Aleppo NGO AFP Sunday 29 Jun 20...,"[{'article': None, 'author': None, 'date': '6/...",CRIME,1


In [11]:
# Loading valid data

valid_df = pd.read_csv("/kaggle/input/crime-split/valid_crime_df.csv")
display(valid_df['label'].value_counts())

valid_df['target'] = valid_df['label'].map(target_map)

valid_df = valid_df.drop(['label'], axis=1)
valid_df = valid_df.rename(columns={'target': 'labels'})

display(valid_df.head())

label
TRUE    42
FAKE    38
Name: count, dtype: int64

,Unnamed: 0,text,metadata,domain,labels
0,528,Russian Defense Ministry: More than 100 ISIS t...,"[{'article': None, 'author': None, 'date': '10...",CRIME,0
1,84,ion 7 April 2017 A salvo of US cruise missiles...,"[{'article': None, 'author': None, 'date': '4/...",CRIME,0
2,615,Over 100 ISIL Terrorists Killed in Russian Off...,"[{'article': None, 'author': None, 'date': '3/...",CRIME,0
3,748,Used Chemical Weapons against Syria Kurds in H...,"[{'article': None, 'author': None, 'date': '7/...",CRIME,1
4,348,5 2017 The jet fighters from the Syrian regime...,"[{'article': None, 'author': None, 'date': '4/...",CRIME,0


In [13]:
# convert to dataset
import datasets
from datasets import Dataset, DatasetDict

train_ds = Dataset.from_pandas(train_df)
valid_ds = Dataset.from_pandas(valid_df)

In [14]:
def get_max_length(model):
    """
    Extracts maximum token length from the model configuration

    :param model: Hugging Face model
    """

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [15]:
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizes dataset batch

    :param batch: Dataset batch
    :param tokenizer: Model tokenizer
    :param max_length: Maximum number of tokens to emit from the tokenizer
    """
    
    encoding = tokenizer(
        batch["text"],
        max_length = max_length,
        truncation = True,
    )

    return encoding

In [16]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):
    """
    Tokenizes dataset for fine-tuning

    :param tokenizer (AutoTokenizer): Model tokenizer
    :param max_length (int): Maximum number of tokens to emit from the tokenizer
    :param seed: Random seed for reproducibility
    :param dataset (str): Instruction dataset
    """

    col_to_delete = ['Unnamed: 0', 'text', 'metadata', 'domain']
    
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    # Apply the preprocessing function and remove the undesired columns
    dataset_tokenized = dataset.map(_preprocessing_function, batched=True, remove_columns=col_to_delete)
    # Rename the target to label as for HugginFace standards
    dataset_tokenized = dataset_tokenized.rename_column("labels", "label")
    # Set to torch format
    dataset_tokenized.set_format("torch")
    
    
#     # Add prompt to each sample
#     print("Preprocessing dataset...")
    
#     # Apply preprocessing to each batch of the dataset & and remove "instruction", "input", "output", and "text" fields
#     _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
#     dataset_tokenized = dataset.map( # call _preprocessing_function on each sample in the dataset
#         _preprocessing_function,
#         batched = True,
#         remove_columns = ['Unnamed: 0', 'text', 'metadata', 'domain']
#     )    

#     # Filter out samples that have "input_ids" exceeding "max_length"
#     dataset_tokenized = dataset_tokenized.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset_tokenized = dataset_tokenized.shuffle(seed = seed)
    
#     dataset_tokenized["labels"] = torch.LongTensor(dataset_tokenized["labels"]) 

    return dataset_tokenized

In [17]:
# preprocessing

# Random seed
seed = 33

# max_length = get_max_length(model)
max_length = 512
preprocessed_train_dataset = preprocess_dataset(tokenizer, max_length, seed, train_ds)
preprocessed_validate_dataset = preprocess_dataset(tokenizer, max_length, seed, valid_ds)


  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [18]:
import numpy as np

np.object = object
display(preprocessed_train_dataset[0])
display(preprocessed_validate_dataset[0])

{'label': tensor(1),
 'input_ids': tensor([    1,   263,   952,   310,  1067,  1161,   267,   297,  8713, 15851,
           360,  2518, 29874,  2598,  2105, 23844, 29925, 24211, 29871, 29906,
         29953,   319,   558, 29871, 29906, 29900, 29896, 29946,  2180,  3203,
         29871, 29947, 29947,   337,  6596,   322, 22384,  8249,   505,  1063,
          9445,   297,  1023,  3841,   310,  1067,  1161,   267,   363,  2761,
           310, 16650,   293, 11840,   297,  8713, 15851, 14841,   360,  2518,
         29874, 12291,   385, 29652,  2318,  1497, 24211, 29889,   450,  8713,
          6392, 21651,  7606,   363, 12968, 26863,  2729,   297, 14933,  1497,
         29871, 29946, 29945, 19626,   285,  1141,  2153,   322, 29871, 29946,
         29941, 22384,  8249,   750,  1063,  9445,   297,   278, 17770,   393,
          4689,   373,   498,  1295,  3250, 29889, 21651,  7606,  8881,   390,
          4479,  1976,  6144, 16790,  1171,  5429, 23844, 29925,   393,   337,
          6596,  8

{'label': tensor(1),
 'input_ids': tensor([    1,   330,   504, 29871, 29906, 29896, 29871, 29906, 29900, 29896,
         29941,   278,  4007,   328, 22384, 17285,  7679,  1691, 10423,   411,
          7745,   262, 10489,   373,   337,  6596, 29899, 29882,  2495, 13491,
           449, 29874, 17692,  9865,  6151,   375, 23393,   472,  3203, 29871,
         29896, 29946, 29900, 29900, 14175,  2638,   550,  3704,  5866,   322,
          4344, 29889, 29871, 29906, 29889,  4367,  3454, 29901, 10504,   408,
          1532,   408, 10387,  5922, 24905,  9571,   278, 16661,   322,   408,
           372,   471, 24067,   263,  2654,  1196, 12061,  8859,   491,  3148,
          7178,  2261,   547,  4250,  3304,   278,  3303,  3900, 13240,   363,
           263,  6035,  3321, 21283,  8401,   967,  1370,  9981,   297,   278,
         25620, 10800,   273, 17649,   304,  8713,  2849, 29889, 29871, 29941,
         29889,   624,  5357,   267, 29901,  1551,  5533, 29871, 29929, 29871,
         29906, 29

In [19]:
print(max(len(preprocessed_train_dataset[i]['input_ids']) for i in range(len(preprocessed_train_dataset))))

512


# Training model

In [20]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 64

# Dropout probability for LoRA layers
lora_dropout = 0.1

# Bias
bias = "none"

# Task type
# task_type = "CAUSAL_LM" # basic
# task_type = "SEQ_CLS" #class
task_type = TaskType.SEQ_CLS

################################################################################
# TrainingArguments parameters
################################################################################


# Batch size per GPU for training
per_device_train_batch_size = 16

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-5

# Optimizer to use
optim = "paged_adamw_32bit"

# Number of training steps (overrides num_train_epochs)
max_steps = -1
train_epochs = 1

# Linear warmup steps from 0 to learning_rate
warmup_steps = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True

# Log every X updates steps
# need to be multiples of gradient_accumulation_steps
logging_steps = 20
eval_steps = 20

In [22]:
def find_all_linear_names(model):
    """
    Find modules to apply LoRA to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            print(name)
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

In [23]:
def find_all_linear_names_new(model):
    """
    Find modules to apply LoRA to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            lora_module_names.add(name)

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

In [21]:
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    """
    Creates Parameter-Efficient Fine-Tuning configuration for the model

    :param r: LoRA attention dimension
    :param lora_alpha: Alpha parameter for LoRA scaling
    :param modules: Names of the modules to apply LoRA to
    :param lora_dropout: Dropout Probability for LoRA layers
    :param bias: Specifies if the bias parameters should be trained
    """
    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

In [22]:
def print_trainable_parameters(model, use_4bit = False):
    """
    Prints the number of trainable parameters in the model.

    :param model: PEFT model
    """

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

In [23]:
def compute_metrics(logits_and_labels):
  logits, labels = logits_and_labels
  predictions = np.argmax(logits, axis=-1)
#   predictions = logits
  acc = np.mean(predictions == labels)
  precision = precision_score(labels, predictions, average='weighted')
  recall = recall_score(labels, predictions, average='weighted')
  f1 = f1_score(labels, predictions, average='weighted')
  gmean = geometric_mean_score(labels, predictions, average='weighted')
  return {'accuracy': acc, 'precision': precision, 'recall': recall, 'f1_score': f1, 'gmean': gmean}

In [ ]:
# is not used

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        display("labels", labels)
        display("labels type", labels.dtype)
        display(self.model.config.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int))
        
        outputs = model(**inputs)
        logits = outputs.logits
#         logits = logits.cpu()
        display("old logits", logits)
        logits = logits.cpu().detach().numpy()
        logits_eager = tf.nn.softmax(logits, name=None)
        new_logits = torch.from_numpy(logits_eager.numpy())
        display("new logits", new_logits)
        
#         display("logits", logits)
#         display("logits-view", logits.view(-1, self.model.config.num_labels))
#         logits = np.argmax(logits, axis=-1)
#         value, index = torch.max(logits, 1)
#         display("logits value", value)
#         value = torch.argmax(logits)
#         display("logits value", value)
#         loss_fct = torch.nn.BCEWithLogitsLoss()
        loss_fct = torch.nn.CrossEntropyLoss()
#         loss = tf.nn.softmax_cross_entropy_with_logits(logits.cpu().detach().numpy(), labels.cpu().detach().numpy(), name=None)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels)
        display("...loss", loss)
#         loss = loss_fct(logits, labels.unsqueeze(1))
        return (loss, outputs) if return_outputs else loss

In [27]:
def fine_tune(model,
          tokenizer,
          train_ds,
          valid_ds,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim):
    """
    Prepares and fine-tune the pre-trained model.

    :param model: Pre-trained Hugging Face model
    :param tokenizer: Model tokenizer
    :param dataset: Preprocessed training dataset
    """

    # Enable gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # Prepare the model for training
    model = prepare_model_for_kbit_training(model)

    # Get LoRA module names
#     target_modules = find_all_linear_names_new(model)
    target_modules = []
    for i in range(7):
        target_modules += [f'model.layers.{i}.self_attn.q_proj',
                      f'model.layers.{i}.self_attn.k_proj',
                      f'model.layers.{i}.self_attn.v_proj',
                      f'model.layers.{i}.self_attn.o_proj', 
                      f'model.layers.{i}.mlp.gate_proj',
                      f'model.layers.{i}.mlp.up_proj',
                      f'model.layers.{i}.mlp.down_proj']

    print(target_modules)

    # Create PEFT configuration for these modules and wrap the model to PEFT
    peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)

    # Training parameters
    trainer = Trainer(
        model = model,
        train_dataset = train_ds,
        eval_dataset = valid_ds,
        compute_metrics=compute_metrics,
        args = TrainingArguments(
            num_train_epochs=train_epochs,
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
            # max_steps = max_steps,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            optim = optim,
#             evaluation_strategy = "steps", # does evaluation during training
            evaluation_strategy = "epoch", # does evaluation at the end of the epoch
            # save_strategy = "steps",
            eval_steps = eval_steps,
            # save_steps = max_steps,
            # report_to="tensorboard"
        ),
        data_collator = DataCollatorWithPadding(tokenizer)
    )

    model.config.use_cache = False

    model = torch.compile(model)

    do_train = True

    # Launch training and log metrics
    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

        # compute evaluation results
        # metrics = trainer.evaluate()
        # # save evaluation results
        # trainer.log_metrics("eval", metrics)
        # trainer.save_metrics("eval", metrics)
        # print(metrics)

    # Save model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok = True)
    trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()

In [28]:
fine_tune(model, tokenizer, preprocessed_train_dataset, preprocessed_validate_dataset, lora_r, lora_alpha, lora_dropout, bias, task_type, per_device_train_batch_size, gradient_accumulation_steps, warmup_steps, max_steps, learning_rate, fp16, logging_steps, output_dir, optim)

['model.layers.0.self_attn.q_proj', 'model.layers.0.self_attn.k_proj', 'model.layers.0.self_attn.v_proj', 'model.layers.0.self_attn.o_proj', 'model.layers.0.mlp.gate_proj', 'model.layers.0.mlp.up_proj', 'model.layers.0.mlp.down_proj', 'model.layers.1.self_attn.q_proj', 'model.layers.1.self_attn.k_proj', 'model.layers.1.self_attn.v_proj', 'model.layers.1.self_attn.o_proj', 'model.layers.1.mlp.gate_proj', 'model.layers.1.mlp.up_proj', 'model.layers.1.mlp.down_proj', 'model.layers.2.self_attn.q_proj', 'model.layers.2.self_attn.k_proj', 'model.layers.2.self_attn.v_proj', 'model.layers.2.self_attn.o_proj', 'model.layers.2.mlp.gate_proj', 'model.layers.2.mlp.up_proj', 'model.layers.2.mlp.down_proj', 'model.layers.3.self_attn.q_proj', 'model.layers.3.self_attn.k_proj', 'model.layers.3.self_attn.v_proj', 'model.layers.3.self_attn.o_proj', 'model.layers.3.mlp.gate_proj', 'model.layers.3.mlp.up_proj', 'model.layers.3.mlp.down_proj', 'model.layers.4.self_attn.q_proj', 'model.layers.4.self_attn.k_

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Training...


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1 Score,Gmean
0,No log,1.514375,0.475000,0.472059,0.475000,0.472348,0.471162


***** train metrics *****
  epoch                    =       0.98
  total_flos               = 11874450GF
  train_loss               =     1.5946
  train_runtime            = 0:20:22.69
  train_samples_per_second =      0.526
  train_steps_per_second   =      0.008
{'train_runtime': 1222.6959, 'train_samples_per_second': 0.526, 'train_steps_per_second': 0.008, 'total_flos': 1.27500936019968e+16, 'train_loss': 1.5945724487304687, 'epoch': 0.98}
Saving last checkpoint of the model...


# SAVING TO HF
*restart, import all dependencies*


In [29]:
label2id = {'TRUE': 0, 'FAKE': 1}
id2label = {0: 'TRUE', 1: 'FAKE'}


# Load fine-tuned weights
model = AutoPeftModelForSequenceClassification.from_pretrained(
    "FakeNewsLlama/CrimeTrueFake7Layer", device_map = "auto", torch_dtype = torch.bfloat16, offload_folder='./', num_labels=2, label2id=label2id, id2label=id2label)

model.config.pad_token_id = model.config.eos_token_id
model.config.problem_type = "single_label_classification"



adapter_config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at meta-llama/Llama-2-7b-hf and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


adapter_model.bin:   0%|          | 0.00/3.10M [00:00<?, ?B/s]

In [34]:
def evaluate_model(model,
          tokenizer,
          train_ds,
          valid_ds,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim):
    """
    Prepares and fine-tune the pre-trained model.

    :param model: Pre-trained Hugging Face model
    :param tokenizer: Model tokenizer
    :param dataset: Preprocessed training dataset
    """
    # Training parameters
    trainer = Trainer(
        model = model,
        train_dataset = train_ds,
#         eval_dataset = valid_ds.shuffle(seed=42).select(range(400)),
        eval_dataset = valid_ds,
        compute_metrics=compute_metrics,
        args = TrainingArguments(
            num_train_epochs=train_epochs,
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
            # max_steps = max_steps,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            optim = optim,
            evaluation_strategy = "steps", # does evaluation during training
            # evaluation_strategy = "epoch", # does evaluation at the end of the epoch
            # save_strategy = "steps",
            eval_steps = eval_steps,
            # save_steps = max_steps,
            # report_to="tensorboard"
        ),
        data_collator = DataCollatorWithPadding(tokenizer)
    )

    model.config.use_cache = False

    model = torch.compile(model)
    torch.backends.cuda.enable_mem_efficient_sdp(False)
    torch.backends.cuda.enable_flash_sdp(False)

    do_train = True

    # Launch training and log metrics
    print("Training...")

    if do_train:
        #         train_result = trainer.train()
        #         metrics = train_result.metrics
        #         trainer.log_metrics("train", metrics)
        #         trainer.save_metrics("train", metrics)
        #         trainer.save_state()
        #         print(metrics)
        # compute evaluation results
        metrics = trainer.evaluate()
        # save evaluation results
        trainer.log_metrics("eval", metrics)
        trainer.save_metrics("eval", metrics)
        print(metrics)

# Save model
# print("Saving last checkpoint of the model...")
# os.makedirs(output_dir, exist_ok = True)
# trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()
    
    return model

In [35]:
evaluate_model(model, tokenizer, preprocessed_train_dataset, preprocessed_validate_dataset, lora_r, lora_alpha, lora_dropout, bias, task_type, per_device_train_batch_size, gradient_accumulation_steps, warmup_steps, max_steps, learning_rate, fp16, logging_steps, output_dir, optim)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Training...


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


***** eval metrics *****
  eval_accuracy           =      0.375
  eval_f1_score           =     0.3719
  eval_gmean              =     0.3784
  eval_loss               =     1.5601
  eval_precision          =     0.3763
  eval_recall             =      0.375
  eval_runtime            = 0:00:36.66
  eval_samples_per_second =      2.182
  eval_steps_per_second   =      0.273
{'eval_loss': 1.5600738525390625, 'eval_accuracy': 0.375, 'eval_precision': 0.37627877237851665, 'eval_recall': 0.375, 'eval_f1_score': 0.3718671679197995, 'eval_gmean': 0.37843042486851586, 'eval_runtime': 36.6684, 'eval_samples_per_second': 2.182, 'eval_steps_per_second': 0.273}


UnboundLocalError: local variable 'model' referenced before assignment

In [ ]:
# Merge the LoRA layers with the base model
model = model.merge_and_unload()

In [ ]:
# only to remove directories
import shutil
# shutil.rmtree('/kaggle/working/news_classification_llama2_7b/final_merged_checkpoint')

In [ ]:
# Save fine-tuned model at a new location
output_merged_dir = "/kaggle/working/news_classification_llama2_7b/final_merged_checkpoint2"
os.makedirs(output_merged_dir, exist_ok = True)
model.save_pretrained(output_merged_dir, safe_serialization = True)

In [ ]:
merged_model = "FakeNewsLlama/CrimeTrueFakeClassifier_C1"
model.push_to_hub(merged_model, use_auth_token = True)

In [ ]:
# Save tokenizer for easy inference
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.save_pretrained(output_merged_dir)

In [ ]:
tokenizer.push_to_hub(merged_model, use_auth_token = True)

# TESTING
*restart only*

In [ ]:
!pip install accelerate==0.27.2 --progress-bar off
!pip install -q peft==0.4.0 --progress-bar off
# !pip install -q bitsandbytes==0.40.2 --progress-bar off
!pip install -q bitsandbytes==0.41.3 --progress-bar off
!pip install -q transformers==4.38.2 --progress-bar off
!pip install -q trl==0.4.7 --progress-bar off

!pip install datasets==2.18.0

In [ ]:
import os
from random import randrange
from functools import partial
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoModelForSequenceClassification,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          DataCollatorWithPadding,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM, AutoPeftModelForSequenceClassification, TaskType
from trl import SFTTrainer

In [ ]:
from transformers import pipeline
from datasets import load_dataset

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
 # Get number of GPU device and set maximum memory
n_gpus = torch.cuda.device_count()
max_memory = f'{40960}MB'
trained_model_name = "FakeNewsLlama/CrimeTrueFakeClassifier_C1"

In [ ]:
test_file_path = "/kaggle/input/crime-fakenews-data/test_crime_df.csv"
test_ds = load_dataset("csv", data_files=test_file_path)["train"]

test_ds_subset = test_ds[:2]
test_ds_subset

In [ ]:
# prompting using validation set
results = pd.DataFrame(columns = ['id', 'text', 'expected_domain', 'predicted_domain'])

gen = pipeline('text-classification', model=trained_model_name)
predicted_ds = gen(test_ds_subset["text"])
# output in form: [{'label': 'TRUE', 'score': 0.6169524788856506},...]

In [ ]:
id = 0
results = pd.DataFrame(columns = ['id', 'text', 'expected_domain', 'predicted_domain'])

for predicted in predicted_ds:
    results.loc[id] = [str(id), test_ds_subset['text'][id], test_ds_subset['label'][id], predicted['label']]
    id += 1

display(results)

In [ ]:
os.makedirs("/kaggle/working/tests/Crime_C1/", exist_ok = True)
results.to_csv("/kaggle/working/tests/Crime_C1/crime_test_results.csv")